<a href="https://colab.research.google.com/github/RohanSaxena14/Sentiment-Analysis/blob/master/Sentiment_analysis_using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Installing required packages**
Read more about Bert-as-server from here

https://bert-as-service.readthedocs.io/en/latest/

In [0]:
#Just initializing the bert's server and client 
!pip install tensorflow==1.15
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

In [0]:
#Downloading and unzipping the pre-trained bert model named "uncased_L-12_H-768_A-12"
#if input is requested type "A"

!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip
!nohup bert-serving-start -model_dir=./uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [0]:
#creating bert Client object

from bert_serving.client import BertClient
bc = BertClient()

In [12]:
import tensorflow

print(tensorflow.__version__)

1.15.0


# **We will be using "IMDB reviews dataset" for training our model.**


1.   **Here we are first loading the dataset using pandas**
2.   **Then we will clean all reviews as they have HTML tags in them.**



In [0]:
import pandas as pd
from bs4 import BeautifulSoup

df = pd.read_csv("IMDB Dataset.csv")


for i in range(50000):
  html = df.iloc[i, 0]
  soup = BeautifulSoup(html)
  text = soup.get_text()
  df.iloc[i, 0] = text

**This is how our data looks like.**

In [14]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


#**We will be converting sentences into vectors using BERT pre-trained model**
**to learn more about BERT, I refer this post here.**

https://www.searchenginejournal.com/bert-explained-what-you-need-to-know-about-googles-new-algorithm/337247/

In [16]:
BERT_vectors = bc.encode(list(df['review']))

print(BERT_vectors.shape)

/usr/local/lib/python3.6/dist-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


(50000, 768)


#**After forming vectors, we now have to prepare numpy arrays which can be used for training and testing purposes**

In [0]:
import numpy as np

labels = list(df["sentiment"])

labels = np.array([x=="positive" for x in labels]).astype(int)


train_vectors = np.array(BERT_vectors[:25000])
test_vectors = np.array(BERT_vectors[25000:])

train_labels = labels[:25000].reshape(25000, 1)
test_labels = labels[25000:]

**Here are the dimensions of our trainig data**

In [18]:
print(train_labels.shape)
print(train_vectors.shape)

(25000, 1)
(25000, 768)


#**We will be using keras for building our neural network**

read more about it from here

https://victorzhou.com/blog/keras-neural-network-tutorial/

In [0]:
import tensorflow as tf

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(200, activation = 'relu', input_dim = train_vectors.shape[1]),
                             #tf.keras.layers.Dense(800, activation = 'relu'),
                             #tf.keras.layers.Dense(800, activation = 'relu'),
                             #tf.keras.layers.Dense(200, activation = 'relu'),
                             tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [0]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [29]:
model.fit(train_vectors, train_labels, epochs = 10, batch_size = 32, validation_split = 0.1)

Train on 22500 samples, validate on 2500 samples
Epoch 1/10
22500/22500 [==============================] - 2s 67us/sample - loss: 0.5762 - acc: 0.6919 - val_loss: 0.5548 - val_acc: 0.7104
Epoch 2/10
22500/22500 [==============================] - 1s 63us/sample - loss: 0.5351 - acc: 0.7211 - val_loss: 0.5422 - val_acc: 0.7156
Epoch 3/10
22500/22500 [==============================] - 1s 63us/sample - loss: 0.5133 - acc: 0.7372 - val_loss: 0.5456 - val_acc: 0.7184
Epoch 4/10
22500/22500 [==============================] - 1s 63us/sample - loss: 0.4938 - acc: 0.7486 - val_loss: 0.5504 - val_acc: 0.7232
Epoch 5/10
22500/22500 [==============================] - 1s 63us/sample - loss: 0.4666 - acc: 0.7723 - val_loss: 0.5574 - val_acc: 0.7132
Epoch 6/10
22500/22500 [==============================] - 1s 62us/sample - loss: 0.4426 - acc: 0.7895 - val_loss: 0.5587 - val_acc: 0.7196
Epoch 7/10
22500/22500 [==============================] - 1s 62us/sample - loss: 0.4098 - acc: 0.8129 - val_loss: 0.5

#**After training for 10 epochs, we are able to achieve 70% accuracy on test data.**

In [25]:
loss, accuracy = model.evaluate(test_vectors, test_labels)
print("Loss :- ", loss)
print("Accuracy :- ", accuracy)

25000/25000 [==============================] - 1s 33us/sample - loss: 0.9592 - acc: 0.6924
Loss :-  0.9591589849090576
Accuracy :-  0.6924


#**Lastly we would like to see, how well a simple Logistic Regressor can fit our data and it turns out that it is achieving slightly greater Accuracy over Neural network model.**

**Note:- It is only because we are using a shallow neural net and its hyper parameters are not tuned very well, so feel free to experiment with it and you will deffinately be able to achieve much higher accuracy.**

In [26]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(train_vectors, train_labels)
score = classifier.score(test_vectors, test_labels)

print("Accuracy:", score)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.70892


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
